In [1]:
import yaml
import copy
import os
import shutil
from oscar_python.client import Client # api de python para OSCAR
import json
import time
from datetime import datetime


In [2]:
directory = 'replicas'

fdl_directory='fedecluster_oscar_replica1_orchestrator.yaml'


# Leer el contenido del archivo YAML y almacenarlo en una variable
with open(fdl_directory, 'r') as file:
    fdl= yaml.safe_load(file)
name = fdl['functions']['oscar'][0]
id_cluster=list(name.keys())[0]
endpoint=fdl['clusters'][id_cluster]['endpoint']
print(endpoint)
auth_user=fdl['clusters'][id_cluster]['auth_user']
auth_password=fdl['clusters'][id_cluster]['auth_password']
ssl_verify=fdl['clusters'][id_cluster]['ssl_verify']
service_name=name[id_cluster]['name']
print(service_name)

client = Client(id_cluster,endpoint, auth_user, auth_password, ssl_verify)

https://condescending-albattani4.im.grycap.net
grayifyr0


In [ ]:
response = client.remove_all_jobs(service_name)
print(response)

In [ ]:

while True:
    time.sleep(10)
    Succeeded=0
    Failed=0
    Pending=0
    i=0
    log_list = client.list_jobs(service_name)
    data = json.loads(log_list.text)
    for log in list(data.keys()):
        i+=1
        log=data[log]['status']  # Accediendo a un valor en el diccionario
        if log =='Succeeded':
            Succeeded+=1
        elif log== 'Failed':
            Failed+=1
        elif log =='Pending':
            Pending+=1
    print(i)
    print(Succeeded)
    print(Failed)
    print(Pending)
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open("valores_job.txt", "a") as archivo:
        archivo.write(f"{timestamp},{i}, {Succeeded}, {Failed}, {Pending}\n")
    


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

def leer_datos_del_archivo(nombre_archivo):
    tiempos = []
    total = []
    succeeded = []
    failed=[]
    pending=[]
    

    with open(nombre_archivo, "r") as archivo:
        for linea in archivo:
            partes = linea.strip().split(",")
            timestamp_str = partes[0]
            
            valor1_str = partes[1].strip()
            valor2_str = partes[2].strip()
            valor3_str = partes[3].strip()
            valor4_str = partes[4].strip()

            # Convertir la marca de tiempo a un objeto datetime
            timestamp = datetime.strptime(timestamp_str, "%Y-%m-%d %H:%M:%S")
            tiempos.append(timestamp)

            # Convertir los valores a enteros
            valor1 = int(valor1_str)
            valor2 = int(valor2_str)
            valor3 = int(valor3_str)
            valor4 = int(valor4_str)

            total.append(valor1)
            succeeded.append(valor2)
            failed.append(valor3)
            pending.append(valor4)

    return tiempos, total, succeeded,failed,pending

def graficar_datos(tiempos, total, succeeded,failed,pending,imagen):
    plt.figure(figsize=(14, 7))

    # Primer gráfico
    plt.subplot(2, 2, 1)
    plt.plot(tiempos, total, label='Total', color='blue')
    plt.xlabel('Time')
    plt.ylabel('Job')
    plt.title('Total Job')
    
    plt.grid(True)

    # Segundo gráfico
    plt.subplot(2, 2, 2)
    plt.plot(tiempos, succeeded, label='Succeeded', color='green')
    plt.xlabel('Time')
    plt.ylabel('Job')
    plt.title('Succeeded Job')
    
    plt.grid(True)
    
    # Tercer gráfico
    plt.subplot(2, 2, 3)
    plt.plot(tiempos, failed, label='Failed', color='green')
    plt.xlabel('Time')
    plt.ylabel('Job')
    plt.title('Failed Job')
    
    plt.grid(True)

    #plt.tight_layout()
    #plt.show()
    # Cuarto gráfico
    plt.subplot(2, 2, 4)
    plt.plot(tiempos, pending, label='Pending', color='green')
    plt.xlabel('Time')
    plt.ylabel('Job')
    plt.title('Pending Job')
    
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(imagen, format='jpg')
    plt.close()

imagen = "analisis_job.jpg"
archivo = "valores_job.txt"
tiempos, total, succeeded,failed,pending = leer_datos_del_archivo(archivo)
graficar_datos(tiempos, total, succeeded,failed,pending, imagen)